In [1]:
# !pip install mediapipe opencv-python

In [2]:
# pip install --upgrade scikit-learn

In [1]:
import mediapipe as mp
import cv2
import numpy as np
import math
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, LayerNormalization, Bidirectional, Dropout, Attention, Input, AdditiveAttention, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import os


# from scikeras.wrappers import KerasClassifier
# # from scikeras.wrappers import KerasClassifier

In [2]:
SEQUENCE_LENGTH= 50
LOOKBACK = 15

In [3]:
def calculate_distance(point1, point2):
    return math.sqrt(
        (point2.x - point1.x) ** 2
        + (point2.y - point1.y) ** 2
        + (point2.z - point1.z) ** 2
    )


def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(
        a[1] - b[1], a[0] - b[0]
    )
    angle = np.abs(radians * 180.0 / np.pi)

    return angle


In [4]:
 def get_features(landmarks):
            ## left body parts values

            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
            left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
            left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
            left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]
            left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]


            ## Right body part values
            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
            right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
            right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
            right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
            right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]
            right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value]



            ## COORDS of left body parts
            left_shoulder_coords = [left_shoulder.x, left_shoulder.y, left_shoulder.z]
            left_elbow_coords = [left_elbow.x, left_elbow.y, left_elbow.z]
            left_wrist_coords = [left_wrist.x, left_wrist.y, left_wrist.z]
            left_hip_coords = [left_hip.x, left_hip.y, left_hip.z]
            left_knee_coords = [left_knee.x, left_knee.y, left_knee.z]
            left_ankle_coords = [left_ankle.x , left_ankle.y , left_ankle.z]

            ## COORDS OF Right Body Parts
            right_shoulder_coords = [ right_shoulder.x , right_shoulder.y , right_shoulder.z ]
            right_elbow_coords = [right_elbow.x, right_elbow.y, right_elbow.z]
            right_wrist_coords = [right_wrist.x, right_wrist.y, right_wrist.z]
            right_hip_coords = [right_hip.x, right_hip.y, right_hip.z]
            right_knee_coords = [right_knee.x , right_knee.y, right_knee.z]
            right_ankle_coords = [right_ankle.x , right_ankle.y , right_ankle.z]

            ## CALCULATING DISTANCES
            left_hip_sh_dist = calculate_distance(left_hip, left_shoulder)
            right_hip_sh_dist = calculate_distance(right_hip, right_shoulder)

            left_wrist_sh_dist = calculate_distance(left_wrist, left_shoulder)
            right_wrist_sh_dist = calculate_distance(right_wrist, right_shoulder)

            left_ch_elbow_dist = calculate_distance(left_shoulder, left_elbow)
            right_ch_elbow_dist = calculate_distance(right_shoulder, right_elbow)

            left_wrist_hip_dist = calculate_distance(left_hip, left_wrist)
            right_wrist_hip_dist = calculate_distance(right_hip, right_wrist)

            left_elbow_wrist_dist = calculate_distance(left_elbow, left_wrist)
            right_elbow_wrist_dist = calculate_distance(right_elbow, right_wrist)

            left_vert_dist_sh_hip = left_shoulder.y - left_hip.y
            right_vert_dist_sh_hip = right_shoulder.y - right_hip.y

            left_horiz_dist_sh_hip = left_shoulder.x - left_hip.x
            right_horiz_dist_sh_hip = right_shoulder.x - right_hip.x

            left_wrist_height = left_shoulder.y - left_wrist.y
            right_wrist_height = right_shoulder.y - right_wrist.y

            ## CALCULATING ANGLES
            left_elbow_angle = calculate_angle(
                left_shoulder_coords, left_elbow_coords, left_wrist_coords
            )
            right_elbow_angle = calculate_angle(
                right_shoulder_coords, right_elbow_coords, right_wrist_coords
            )

            left_hip_angle = calculate_angle(
                left_shoulder_coords, left_hip_coords, left_knee_coords
            )
            right_hip_angle = calculate_angle(
                right_shoulder_coords, right_hip_coords, right_knee_coords
            )
            left_shoulder_flexion_extension_angle = calculate_angle(
                left_hip_coords, left_shoulder_coords, left_elbow_coords
            )
            right_shoulder_flexion_extension_angle = calculate_angle(
                right_hip_coords, right_shoulder_coords, right_elbow_coords
            )

            # Shoulder Abduction/Adduction
            left_shoulder_abduction_adduction_angle = calculate_angle(
                left_shoulder_coords, left_hip_coords, left_knee_coords
            )
            right_shoulder_abduction_adduction_angle = calculate_angle(
                right_shoulder_coords, right_hip_coords, right_knee_coords
            )

            # Body Tilt Angle (Tilt of the body relative to vertical axis)
            left_body_tilt_angle = calculate_angle(
                [left_shoulder.x, left_shoulder.y], [left_hip.x, left_hip.y], [0, 1]
            )
            right_body_tilt_angle = calculate_angle(
                [right_shoulder.x, right_shoulder.y], [right_hip.x, right_hip.y], [0, 1]
            )

            ## Ankle to Shoulder angle
            left_ankle_shoulder_angle = calculate_angle(
                left_shoulder_coords, left_hip_coords, left_ankle_coords
            )
            right_ankle_shoulder_angle = calculate_angle(
                right_shoulder_coords, right_hip_coords, right_ankle_coords
            )

            ## Ankle Shoulder Distance
            left_ankle_shoulder_distance = calculate_distance(
                left_ankle , left_shoulder
            )
            right_ankle_shoulder_distance = calculate_distance(
                right_ankle , right_shoulder
            )

            ## Ankle knee angle
            left_ankle_knee_angle = calculate_angle(
                left_knee_coords, left_hip_coords, left_ankle_coords
            )
            right_ankle_knee_angle = calculate_angle(
                right_knee_coords, right_hip_coords, right_ankle_coords
            )

            ## Elbow flexion angle
            left_elbow_flexion_angle = calculate_angle(left_shoulder_coords , left_wrist_coords , left_elbow_coords)
            right_elbow_flexion_angle = calculate_angle(right_shoulder_coords , right_wrist_coords , right_elbow_coords)


            return (
                right_elbow_flexion_angle,
                left_elbow_flexion_angle,
                left_ankle_knee_angle,
                right_ankle_shoulder_distance,
                right_ankle_knee_angle,
                left_ankle_shoulder_distance,
                right_ankle_shoulder_angle,
                left_ankle_shoulder_angle,
                left_elbow_angle,
                right_elbow_angle,
                left_hip_angle,
                right_hip_angle,
                left_shoulder_flexion_extension_angle,
                right_shoulder_flexion_extension_angle,
                left_shoulder_abduction_adduction_angle,
                right_shoulder_abduction_adduction_angle,
                left_body_tilt_angle,
                right_body_tilt_angle,
                left_hip_sh_dist,
                right_hip_sh_dist,
                left_wrist_sh_dist,
                right_wrist_sh_dist,
                left_ch_elbow_dist,
                right_ch_elbow_dist,
                left_wrist_hip_dist,
                right_wrist_hip_dist,
                left_elbow_wrist_dist,
                right_elbow_wrist_dist,
                left_vert_dist_sh_hip,
                right_vert_dist_sh_hip,
                left_horiz_dist_sh_hip,
                right_horiz_dist_sh_hip,
                left_wrist_height,
                right_wrist_height,
            )

In [5]:
def generate_sequences_with_lookback(df, seq_length, lookback):
    # Load CSV file
    features = df.values
    sequences = []

    for i in range(0, len(features), seq_length- lookback):
      ## @TODO use a better way
      if i+seq_length>len(features):
        break
      sequence = features[i:i+seq_length]
      sequences.append(sequence)
    return (sequences)

In [6]:
from tensorflow.keras.models import load_model
model = load_model("models/Model-1.keras")
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_10 (LSTM)                       │ (None, 50, 64)              │          25,344 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer_normalization_10               │ (None, 50, 64)              │             100 │
│ (LayerNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_11 (LSTM)                       │ (None, 64)                  │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer_normalization_11               │ (None, 64)                  │             128 │
│ (LayerNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 4)                   │             260 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 189,050 (738.48 KB)

 Trainable params: 63,016 (246.16 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 126,034 (492.32 KB)

In [7]:
# labels_map = ['bench press', 'bicep curl', 'push-up', 'rest']
labels_map = ['bench press', 'bicep curl', 'push-up', 'sitting', 'standing','walking']
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
x, y = 0, 30
background_color = (0, 0, 0)
def get_features_from_video_frame_skip(video_path):
    
    cap = cv2.VideoCapture(video_path)
    features_list = []

    prediction = None

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
                
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            

            text = f"Predicted Label: {prediction}"
            (text_width, text_height), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX , 1,  2 )
            cv2.rectangle(frame, (x, y - text_height - 10), (x + text_width + 10, y + 10), background_color, cv2.FILLED)
            
            cv2.putText(frame, text, 
                        org =(0, 30) ,
                        fontFace = cv2.FONT_HERSHEY_SIMPLEX ,
                        fontScale = 1,
                        color=(255, 255, 255),
                        thickness=2)

           
            
            cv2.imshow('Webcam Video', frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

            # Make detection
            results = pose.process(image)

            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
                (
                 right_elbow_flexion_angle,
                left_elbow_flexion_angle,
                left_ankle_knee_angle,
                right_ankle_shoulder_distance,
                right_ankle_knee_angle,
                left_ankle_shoulder_distance,
                right_ankle_shoulder_angle,
                left_ankle_shoulder_angle,
                left_elbow_angle,
                right_elbow_angle,
                left_hip_angle,
                right_hip_angle,
                left_shoulder_flexion_extension_angle,
                right_shoulder_flexion_extension_angle,
                left_shoulder_abduction_adduction_angle,
                right_shoulder_abduction_adduction_angle,
                left_body_tilt_angle,
                right_body_tilt_angle,
                left_hip_sh_dist,
                right_hip_sh_dist,
                left_wrist_sh_dist,
                right_wrist_sh_dist,
                left_ch_elbow_dist,
                right_ch_elbow_dist,
                left_wrist_hip_dist,
                right_wrist_hip_dist,
                left_elbow_wrist_dist,
                right_elbow_wrist_dist,
                left_vert_dist_sh_hip,
                right_vert_dist_sh_hip,
                left_horiz_dist_sh_hip,
                right_horiz_dist_sh_hip,
                left_wrist_height,
                right_wrist_height,
                ) = get_features(landmarks)
            
                features_list.append(
                [
                right_elbow_flexion_angle,
                left_elbow_flexion_angle,
                left_ankle_knee_angle,
                right_ankle_shoulder_distance,
                right_ankle_knee_angle,
                left_ankle_shoulder_distance,
                right_ankle_shoulder_angle,
                left_ankle_shoulder_angle,
                left_elbow_angle,
                right_elbow_angle,
                left_hip_angle,
                right_hip_angle,
                left_shoulder_flexion_extension_angle,
                right_shoulder_flexion_extension_angle,
                left_shoulder_abduction_adduction_angle,
                right_shoulder_abduction_adduction_angle,
                left_body_tilt_angle,
                right_body_tilt_angle,
                left_hip_sh_dist,
                right_hip_sh_dist,
                left_wrist_sh_dist,
                right_wrist_sh_dist,
                left_ch_elbow_dist,
                right_ch_elbow_dist,
                left_wrist_hip_dist,
                right_wrist_hip_dist,
                left_elbow_wrist_dist,
                right_elbow_wrist_dist,
                left_vert_dist_sh_hip,
                right_vert_dist_sh_hip,
                left_horiz_dist_sh_hip,
                right_horiz_dist_sh_hip,
                left_wrist_height,
                right_wrist_height,
                ]
            )
            if len(features_list) == SEQUENCE_LENGTH:
                  df = pd.DataFrame(features_list, columns=[
                                              "right_elbow_flexion_angle",
                                              "left_elbow_flexion_angle",
                                              "left_ankle_knee_angle",
                                              "right_ankle_shoulder_distance",
                                              "right_ankle_knee_angle",
                                              "left_ankle_shoulder_distance",
                                              "right_ankle_shoulder_angle",
                                              "left_ankle_shoulder_angle",
                                              "left_elbow_angle",
                                              "right_elbow_angle",
                                              "left_hip_angle",
                                              "right_hip_angle",
                                              "left_shoulder_flexion_extension_angle",
                                              "right_shoulder_flexion_extension_angle",
                                              "left_shoulder_abduction_adduction_angle",
                                              "right_shoulder_abduction_adduction_angle",
                                              "left_body_tilt_angle",
                                              "right_body_tilt_angle",
                                              "left_hip_sh_dist",
                                              "right_hip_sh_dist",
                                              "left_wrist_sh_dist",
                                              "right_wrist_sh_dist",
                                              "left_ch_elbow_dist",
                                              "right_ch_elbow_dist",
                                              "left_wrist_hip_dist",
                                              "right_wrist_hip_dist",
                                              "left_elbow_wrist_dist",
                                              "right_elbow_wrist_dist",
                                              "left_vert_dist_sh_hip",
                                              "right_vert_dist_sh_hip",
                                              "left_horiz_dist_sh_hip",
                                              "right_horiz_dist_sh_hip",
                                              "left_wrist_height",
                                              "right_wrist_height"
                                            ]
                                             )

                  sequences = generate_sequences_with_lookback(df, SEQUENCE_LENGTH , LOOKBACK)
                  # print(np.array(len(sequences)))
                     
                  arr_sequences = np.array(sequences)
                  scaler = MinMaxScaler()
                  num_samples, sequence_length, num_features = arr_sequences.shape
                  arr_reshaped = arr_sequences.reshape(-1, num_features)
      
                  # Fit and transform
                  X_normalized = scaler.fit_transform(arr_reshaped)
                  X_normalized = X_normalized.reshape(num_samples, sequence_length, num_features)

                  preds = model.predict(X_normalized)
                  print(preds)

                  pred_labels = []
                  for pred in preds:
                      i = pred.argmax()
                      prediction = labels_map[i]
                      pred_labels.append(labels_map[i])

                  # print(pred_labels)
                  features_list = []

    cap.release()

In [8]:
get_features_from_video_frame_skip(0)

D:\UNIVERSITY\Final Year Project\AI\env\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 700ms/step
[[0.94650865 0.04106925 0.00456169 0.00786038]]


D:\UNIVERSITY\Final Year Project\AI\env\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
[[0.44379628 0.02316794 0.5273736  0.00566223]]


D:\UNIVERSITY\Final Year Project\AI\env\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


...

D:\UNIVERSITY\Final Year Project\AI\env\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
[[0.976943   0.01786436 0.00146212 0.00373052]]


D:\UNIVERSITY\Final Year Project\AI\env\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
[[0.07470207 0.8994957  0.0201283  0.00567384]]


D:\UNIVERSITY\Final Year Project\AI\env\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [11]:
import warnings
warnings.filterwarnings('ignore')


def evaluate_model(model):
    count_correct = 0
    total = 0
    videos_correctly_classified = 0
    total_vids = 0
    for folder in all_vids_folder:
  
        folder_name = f"test-vids/{folder}"
        
        files_in_folder = os.listdir(folder_name)
        

        for file in files_in_folder:
            try:
                if file.startswith("."):
                     continue
                file_name = f"{folder_name}/{file}"      
                print(f"-----> {file_name} <------")
                preds = make_predictions_for_video(file_name, model)
            # print(preds1, preds2)
                preds_len = len(preds)
                total += preds_len
                count_correct += preds.count(folder)
                if preds.count(folder)/preds_len >=0.5:
                    videos_correctly_classified+=1
                total_vids+=1
                
              
              
            except Exception as e:
                print(file,e, "error there")
    acc = (count_correct/total)*100
    return (count_correct, acc, videos_correctly_classified)

In [12]:
import warnings
warnings.filterwarnings('ignore')


def evaluate_model(model):
    count_correct = 0
    total = 0
    videos_correctly_classified = 0
    total_vids = 0
    for folder in all_vids_folder:
  
        folder_name = f"test-vids/{folder}"
        
        files_in_folder = os.listdir(folder_name)
        

        for file in files_in_folder:
            try:
                if file.startswith("."):
                     continue
                file_name = f"{folder_name}/{file}"      
                print(f"-----> {file_name} <------")
                preds = make_predictions_for_video(file_name, model)
            # print(preds1, preds2)
                preds_len = len(preds)
                total += preds_len
                count_correct += preds.count(folder)
                if preds.count(folder)/preds_len >=0.5:
                    videos_correctly_classified+=1
                total_vids+=1
                
              
              
            except Exception as e:
                print(file,e, "error there")
    acc = (count_correct/total)*100
    return (count_correct, acc, videos_correctly_classified)